In [1]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [36]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [43]:
gpath = "../../graphs/";

In [45]:
graphs = readdir("../../graphs");

In [ ]:
for cur in graphs
    path = gpath * cur * "/"
    
    try
        a = readFromFile(path * "graph.mtx");
        x = readFromFile(path * "x.vec");
        
        tree = []
        for ind in 1:5
            tr = readFromFile(path * "tree$(ind).mtx");
            
            u,v,w = findnz(tr)
            for i in 1:length(w)
                w[i] = a[u[i],v[i]]
            end
            tr = sparse(u,v,w)
            
            push!(tree, tr);
        end
        
        la = lap(a);
        x = x - mean(x)
        b = la * x; b = b - mean(b);
        
        numIts = 200;
        @time for i in 4:5
            @time F = treeSolver(tree[i])
            @time F(b);

            # a run on standard Float64 data types
            @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
            norm(lap(a) * myx64 - b) / norm(b)
            insert!(dbg, 1, "$(numIts) iterations")
            insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
            writeToFile(path * "_log_julia_tree$(i)_default.txt", dbg)

            # now useing higer precision data types
            for hp in [64,128,256,512,1024]
                set_bigfloat_precision(hp)
                laHP = toHighPrecision(la; precision=hp)
                treeHP = toHighPrecision(tree[i]; precision=hp)
                xHP = toHighPrecision(x; precision=hp);
                bHP = toHighPrecision(b; precision=hp);
                FHP = treeSolver(treeHP)

                @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=false);
                insert!(dbg, 1, "$(numIts) iterations")
                insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
                writeToFile(path * "_log_julia_tree$(i)_$(hp).txt", dbg)
            end
        end
            
        println("finished running on ", cur)
    catch
        println("skipping ", cur)
    end
end

  0.019187 seconds (50.18 k allocations: 5.763 MB)
  0.006692 seconds (197.65 k allocations: 3.627 MB)
  1.790024 seconds (39.74 M allocations: 958.480 MB, 6.35% gc time)
117.073208 seconds (476.14 M allocations: 12.103 GB, 34.32% gc time)
122.622728 seconds (476.14 M allocations: 13.262 GB, 35.43% gc time)
130.974267 seconds (476.14 M allocations: 15.579 GB, 35.33% gc time)
179.037261 seconds (476.14 M allocations: 20.214 GB, 35.22% gc time)
385.090883 seconds (476.14 M allocations: 29.484 GB, 33.38% gc time)
  0.076399 seconds (50.18 k allocations: 5.763 MB)
  0.020343 seconds (197.90 k allocations: 3.631 MB)
  4.220791 seconds (39.79 M allocations: 959.265 MB, 15.55% gc time)
256.309432 seconds (476.19 M allocations: 12.104 GB, 34.33% gc time)
280.849614 seconds (476.19 M allocations: 13.263 GB, 35.92% gc time)
313.060384 seconds (476.19 M allocations: 15.580 GB, 36.55% gc time)
251.446783 seconds (476.19 M allocations: 20.215 GB, 35.90% gc time)
307.733533 seconds (476.19 M allocat